In [2]:
import numpy as np
import nlopt
import pandas as pd
from math import log, exp
from multiprocessing import Pool, cpu_count
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#need path to file (it's a big one)
fileIn = "all_chr_2000_discrete_genome_gc.tsv"


In [4]:
def calc_single_dist_rho_only(x,r):
    d,h0,h1,h2,th = x
    r = r*(d)
    calcH0 =(18+13*r + r**2 + 36*th + 22*th**2 + 4*th**3 + r*(6*th+th**2))/((1+th)*(18+13*r+r**2+54*th + 40*th**2+8*th**3+r*(r*th+19*th+6*th**2)))
    calcH2 = (th**2*(36+14*r+r**2+36*th+6*th*r+8*th**2))/((1+th)*(18+13*r+r**2+54*th + 40*th**2+8*th**3+r*(r*th+19*th+6*th**2)))
    calcH1 = 1 - calcH0 - calcH2
    
    val = h0*log(calcH0) + h1*log(calcH1)+h2*log(calcH2)
    return(val)

In [5]:
def single_chr_obj_fun(r,arr):
    rbp = r
    test = np.apply_along_axis(lambda x: calc_single_dist_rho_only(x,rbp),1,arr)
    obfun = np.sum(test)
    #test = [calc_single_dist(x,rbp,g,L) for x in arr]
    #obfun = sum(test)
    return(obfun)

In [6]:
def across_chr_obj_fun(x,dfArrList,numChrs):
    
    rList = x[0:numChrs]
    
    obFun = sum([single_chr_obj_fun(rs,arr) for (rs,arr) in list(zip(rList,dfArrList))])
    return(obFun)

In [7]:
def combine_across_individuals(thisDF):
    test = thisDF
    numSamples = len(test['sample'].unique())
    test = test.groupby(['Unnamed: 0'],as_index=False).sum()
    test['H'] = test['H']/numSamples
    return(test.iloc[:,[0,4,5,6,11]].to_numpy())
    #return( test.iloc[:,[0,4,5,6,11]].to_numpy() )

In [8]:
def find_max_like_group_ind(nchros,rep):
    
    numberChromosomes = nchros
    dfListAuto = [ df.loc[(df['chromosome']== x) & (df['replicate']==rep) & (df['Unnamed: 0']<=1000) &  (df['Unnamed: 0']>= 100)] for x in list(range(1,numberChromosomes+1))]
    dfListAuto = [combine_across_individuals(x) for x in dfListAuto]

    lowerBounds = [0.0001 for x in range(numberChromosomes)]
    upperBounds = [0.1 for x in range(numberChromosomes)]
    startPoints = [0.001 for x in range(numberChromosomes)]


    opt=nlopt.opt(nlopt.LN_NELDERMEAD,numberChromosomes)

    opt.set_lower_bounds(lowerBounds)
    opt.set_upper_bounds(upperBounds)

    opt.set_max_objective(lambda x,grad: across_chr_obj_fun(x,dfListAuto,numberChromosomes))

    res = opt.optimize(startPoints)
    return(res)

In [11]:
def combine_across_individuals_and_replicates(thisDF):
    test = thisDF
    numSamples = len(test['sample'].unique())
    numReplicates = len(test['replicate'].unique())
    test = test.groupby(['Unnamed: 0'],as_index=False).sum()
    test['H'] = test['H']/(numSamples*numReplicates)
    return(test.iloc[:,[0,4,5,6,11]].to_numpy())
    #return( test.iloc[:,[0,4,5,6,11]].to_numpy() )

In [12]:
df = pd.read_csv(fileIn,delimiter='\t')
df = df.loc[ df['chromosome'] != 'X']
df = df.astype({'chromosome':'int64'})

In [13]:
df

,Unnamed: 0,sample,replicate,chromosome,h_0,h_1,h_2,total,h_0_frac,h_1_frac,h_2_frac,H
0,1,1,1,10,52300306,733406,4946,53038658,0.986079,0.013828,0.000093,0.007007
1,2,1,1,10,52300285,733446,4926,53038657,0.986079,0.013829,0.000093,0.007007
2,3,1,1,10,52300265,733484,4907,53038656,0.986078,0.013829,0.000093,0.007007
3,4,1,1,10,52300472,733069,5114,53038655,0.986082,0.013821,0.000096,0.007007
4,5,1,1,10,52300384,733243,5027,53038654,0.986081,0.013825,0.000095,0.007007
...,...,...,...,...,...,...,...,...,...,...,...,...
37999995,1996,10,100,9,45941047,642166,2865,46586078,0.986154,0.013785,0.000061,0.006954
37999996,1997,10,100,9,45940956,642346,2775,46586077,0.986152,0.013788,0.000060,0.006954
37999997,1998,10,100,9,45940797,642662,2617,46586076,0.986149,0.013795,0.000056,0.006954
37999998,1999,10,100,9,45940891,642472,2712,46586075,0.986151,0.013791,0.000058,0.006954


In [14]:
rep = 11
numChrs = 19
coEstimatesAutosomes = find_max_like_group_ind(numChrs,rep)
coEstimatesAutosomes

array([0.0026337 , 0.00359571, 0.00357564, 0.00261904, 0.00338541,
       0.00297479, 0.0030067 , 0.00420282, 0.00336432, 0.00395792,
       0.00301397, 0.00316711, 0.00415027, 0.00365367, 0.00252353,
       0.00361901, 0.0031269 , 0.00354406, 0.00507593])